In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar archivos desde tu equipo
df_agentes = pd.read_excel(r"C:\Users\Camilo\OneDrive - UCompensar\2. SEMINARIO DE GRADO\Data\Agentes_lista.xlsx")
df_alertas_tmo = pd.read_excel(r"C:\Users\Camilo\OneDrive - UCompensar\2. SEMINARIO DE GRADO\Data\Agentes_con_alertas_TMO.xlsx")
df_alertas_creditos = pd.read_excel(r"C:\Users\Camilo\OneDrive - UCompensar\2. SEMINARIO DE GRADO\Data\Creditos_Sospechosos.xlsx")
df_alertas_satisfaccion = pd.read_excel(r"C:\Users\Camilo\OneDrive - UCompensar\2. SEMINARIO DE GRADO\Data\Satisfaccion_Sospechosa.xlsx")
df_alertas_colgadas = pd.read_excel(r"C:\Users\Camilo\OneDrive - UCompensar\2. SEMINARIO DE GRADO\Data\Colgadas_con_credito.xlsx")

# -------------------- Consolidación de alertas --------------------
# TMO
alertas_tmo = df_alertas_tmo[['Agente', 'Cantidad_alertas']].rename(columns={'Cantidad_alertas': 'Alertas_TMO'})

# Créditos sospechosos
alertas_creditos = df_alertas_creditos['Id de empleado'].value_counts().reset_index()
alertas_creditos.columns = ['Agente', 'Alertas_Creditos']

# Satisfacción
alertas_satisfaccion = df_alertas_satisfaccion['Id de empleado'].value_counts().reset_index()
alertas_satisfaccion.columns = ['Agente', 'Alertas_Satisfaccion']

# Colgadas con crédito posterior
alertas_colgadas = df_alertas_colgadas['Id de empleado'].value_counts().reset_index()
alertas_colgadas.columns = ['Agente', 'Alertas_Colgadas']

# Unir todas las alertas
df_alertas = alertas_tmo.merge(alertas_creditos, on='Agente', how='outer') \
                        .merge(alertas_satisfaccion, on='Agente', how='outer') \
                        .merge(alertas_colgadas, on='Agente', how='outer') \
                        .fillna(0)

# -------------------- Score acumulativo de riesgo --------------------
df_alertas['Score_Riesgo'] = (
    df_alertas['Alertas_TMO'] * 1 +
    df_alertas['Alertas_Creditos'] * 1.5 +
    df_alertas['Alertas_Satisfaccion'] * 1.2 +
    df_alertas['Alertas_Colgadas'] * 2
)

# -------------------- Unión con perfil del agente --------------------
df_agentes.rename(columns={'Id de Empleado': 'Agente'}, inplace=True)
df_perfilado = pd.merge(df_alertas, df_agentes, on='Agente', how='left')

# Calcular antigüedad
df_perfilado['Fecha de contratación'] = pd.to_datetime(df_perfilado['Fecha de contratación'])
df_perfilado['Antigüedad (meses)'] = (
    (pd.Timestamp.now() - df_perfilado['Fecha de contratación']) / pd.Timedelta(days=30)
).round()

# -------------------- Visualizaciones --------------------
# Top 10 Score
plt.figure(figsize=(12, 6))
top_riesgo = df_perfilado.sort_values('Score_Riesgo', ascending=False).head(10)
sns.barplot(data=top_riesgo, x='Agente', y='Score_Riesgo')
plt.title('Top 10 agentes con mayor Score de Riesgo')
plt.xlabel('Agente')
plt.ylabel('Score de Riesgo Acumulado')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Distribución por campaña
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_perfilado, x='Campaña', y='Score_Riesgo')
plt.title('Distribución del Score de Riesgo por Campaña')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# -------------------- Exportar a Excel --------------------
df_perfilado.to_excel(r"C:\Users\Camilo\OneDrive - UCompensar\2. SEMINARIO DE GRADO\Data\Perfil_Riesgo_Agentes.xlsx", index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Camilo\\OneDrive - UCompensar\\2. SEMINARIO DE GRADO\\Data\\Agentes_con_alertas_TMO.xlsx'